In [1]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit

from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV

from sklearn.cluster import KMeans

warnings.filterwarnings('ignore')


/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


基本変数定義

In [2]:
sampling_flg=0 #サンプリング有無をコントロール
submit_flg=1 #保存するかをコントロール（サンプリングしない時のみ）

SEED=12345
sample_num=1000
fold_num=5

#train関連
train_dir='../../01_input/train.csv'
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_dir='../../01_input/test.csv'
test_drop_col=['ID_code']

#結果ファイル関連　nameは自分の名前に変更する
train_preds_dir='../../03_predict_train/name_200_LightGBM_train.csv'
test_preds_dir='../../04_predict_test/name_200_LightGBM_submission.csv'
save_col_name='oof_xgb'

sample_submission_dir='../../01_input/sample_submission.csv'
submission_target_col_name='target'
submission_id_col_name='ID_code'

前処理

In [3]:
#ロード
train_df=pd.read_csv(train_dir)
test_df=pd.read_csv(test_dir)

In [4]:
#サンプリング
if sampling_flg ==1:
    train_df=train_df.sample(n=sample_num,random_state=SEED)
    test_df=test_df.sample(n=sample_num,random_state=SEED)
    train_df=train_df.reset_index()
    test_df=test_df.reset_index()

In [5]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)

In [6]:
predict_col=x_train.columns

モデル実行

In [7]:
clf=KMeans(n_clusters=10, init='k-means++', n_init=10, max_iter=300,
                               tol=0.0001,precompute_distances='auto', verbose=0,
                               random_state=11111, copy_x=True, n_jobs=1)
clf.fit(x_train)
pred=clf.predict(x_test)
x_test['kmeans']=pred
pred=clf.predict(x_train)
x_train['kmeans']=pred

In [8]:
x_test

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,kmeans
0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197,2
1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760,8
2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794,3
3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108,6
4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846,5
5,5.9862,-2.2913,8.6058,7.0685,14.2465,-8.6761,4.2467,14.7632,1.8790,7.2842,...,7.1178,-0.4249,8.8781,14.9438,-2.2151,-6.0233,9.8117,17.1127,10.8240,7
6,8.4624,-6.1065,7.3603,8.2627,12.0104,-7.2073,4.1670,13.0809,-4.3004,6.3181,...,6.8661,4.0971,8.8484,17.5010,0.0295,7.7443,9.1509,18.4736,5.1499,5
7,17.3035,-2.4212,13.3989,8.3998,11.0777,9.6449,5.9596,17.8477,-4.8068,7.4643,...,4.4214,0.9303,1.4994,15.2648,-1.7931,6.5316,10.4855,23.4631,0.7283,4
8,6.9856,0.8402,13.7161,4.7749,8.6784,-13.7607,4.3386,14.5843,2.5883,7.2215,...,7.8754,2.4698,-0.0362,16.7144,0.1221,-1.4328,9.9207,16.9865,-3.3304,7
9,10.3811,-6.9348,14.6690,9.0941,11.9058,-10.8018,3.4508,20.2816,-1.4112,6.7401,...,11.0723,0.8907,4.7680,15.1425,0.6075,-4.4447,9.5788,15.8146,9.3457,6


In [9]:
#model_param
param = {
    "objective" : "binary", 
    "boost":"gbdt",
    "metric":"auc",
    "boost_from_average":"false",
    "num_threads":28,
    "learning_rate" : 0.01,
    "num_leaves" : 13,
    "max_depth":-1,
    "tree_learner" : "serial",
    "feature_fraction" : 0.05,
    "bagging_freq" : 5,
    "bagging_fraction" : 0.4,
    "min_data_in_leaf" : 80,
    "min_sum_hessian_in_leaf" : 10.0,
    "verbosity" : 1,
    'seed': 44000,
    }

In [10]:
def lgbm(train_x,train_y,test_x,test_y):
    trn_data = lgb.Dataset(train_x,train_y)
    val_data = lgb.Dataset(test_x,test_y)
    
    num_round = 1000000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 3000)
    
    val_pred = clf.predict(test_x, num_iteration=clf.best_iteration)
    auc_score=roc_auc_score(test_y, val_pred)
    print("")
    print("AUC = {}".format(auc_score))
    
    return auc_score,clf

In [11]:
def best_clf(pred_x,pred_y,other_x,other_y):
    print("")
    print('pred_num='+str(len(pred_x)))
    print('other_num='+str(len(other_x)))
    max_score=0
    best_i=0
    print("")
    for i in range(0,3):
        #model_param
        param = {
            "objective" : "binary", 
            "boost":"gbdt",
            "metric":"auc",
            "boost_from_average":"false",
            "num_threads":28,
            "learning_rate" : 0.01,
            "num_leaves" : 13,
            "max_depth":-1,
            "tree_learner" : "serial",
            "feature_fraction" : 0.05,
            "bagging_freq" : 5,
            "bagging_fraction" : 0.4,
            "min_data_in_leaf" : 80,
            "min_sum_hessian_in_leaf" : 10.0,
            "verbosity" : 1,
            'seed': i,
            }
        sample_num=int(len(other_x)*0.9)
        print("")
        print('sample_num='+str(sample_num))
        other_sample_x=other_x.sample(n=sample_num,random_state=i)
        other_sample_y=other_y.sample(n=sample_num,random_state=i)
        score,model=lgbm(other_sample_x,other_sample_y,pred_x,pred_y)
        if max_score < score :
            max_score=score
            pred_model=model
            best_i=i
    print("")
    print('Best Model:'+str(i)+" Best Score:"+str(max_score))
    return pred_model

In [12]:
x_train['target']=y_train
pred_x=x_train.query('kmeans == 0')
other_x=x_train.query('not kmeans == 0')
pred_y=pred_x['target']
other_y=other_x['target']
pred_x=pred_x.drop('target',axis=1)
other_x=other_x.drop('target',axis=1)

In [13]:
y_test = np.zeros((len(x_test), 1))
y_train_preds = np.zeros((len(x_train), 1))
for i in range(0,10):
    print("")
    print("kmeans:"+str(i))
    x_train['target']=y_train
    pred_x=x_train.query('kmeans =='+str(i))
    other_x=x_train.query('not kmeans =='+str(i))
    pred_y=pred_x['target']
    other_y=other_x['target']
    pred_x=pred_x.drop('target',axis=1)
    other_x=other_x.drop('target',axis=1)
    
    best_model=best_clf(pred_x,pred_y,other_x,other_y)
    
    x_test_pred=x_test[x_test['kmeans']==i]
    
    y_test_pred=best_model.predict(x_test_pred)
    y_train_pred=best_model.predict(pred_x)
    
    x_test_pred_index=x_test_pred.index
    x_train_pred_index=pred_x.index
    
    y_test[x_test_pred_index, :]=y_test_pred.reshape((-1, 1))
    y_train_preds[x_train_pred_index, :]=y_train_pred.reshape((-1, 1))

print("Over All AUC = {}".format(roc_auc_score(y_train, y_train_preds)))
    


kmeans:0

pred_num=20763
other_num=179237


sample_num=161313
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.901769	valid_1's auc: 0.881983
[2000]	training's auc: 0.913408	valid_1's auc: 0.89058
[3000]	training's auc: 0.921125	valid_1's auc: 0.895256
[4000]	training's auc: 0.92718	valid_1's auc: 0.898192
[5000]	training's auc: 0.932299	valid_1's auc: 0.899885
[6000]	training's auc: 0.936735	valid_1's auc: 0.900649
[7000]	training's auc: 0.941023	valid_1's auc: 0.900961
[8000]	training's auc: 0.944909	valid_1's auc: 0.901262
[9000]	training's auc: 0.948601	valid_1's auc: 0.901544
[10000]	training's auc: 0.952124	valid_1's auc: 0.901436
[11000]	training's auc: 0.955533	valid_1's auc: 0.901393
[12000]	training's auc: 0.958782	valid_1's auc: 0.901492
Early stopping, best iteration is:
[9099]	training's auc: 0.948953	valid_1's auc: 0.901594

AUC = 0.9015939773459362

sample_num=161313
Training until validation scores don't improve for 3000 rounds.


[9000]	training's auc: 0.948054	valid_1's auc: 0.89957
[10000]	training's auc: 0.951536	valid_1's auc: 0.899633
[11000]	training's auc: 0.954889	valid_1's auc: 0.8996
[12000]	training's auc: 0.958105	valid_1's auc: 0.899555
[13000]	training's auc: 0.961164	valid_1's auc: 0.899514
[14000]	training's auc: 0.964096	valid_1's auc: 0.899207
Early stopping, best iteration is:
[11185]	training's auc: 0.955493	valid_1's auc: 0.899702

AUC = 0.8997015523442671

sample_num=162869
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.9013	valid_1's auc: 0.883276
[2000]	training's auc: 0.913015	valid_1's auc: 0.890322
[3000]	training's auc: 0.920884	valid_1's auc: 0.89404
[4000]	training's auc: 0.926874	valid_1's auc: 0.89631
[5000]	training's auc: 0.932027	valid_1's auc: 0.897613
[6000]	training's auc: 0.936572	valid_1's auc: 0.898706
[7000]	training's auc: 0.940799	valid_1's auc: 0.899087
[8000]	training's auc: 0.944698	valid_1's auc: 0.899325
[9000]	training's

[8000]	training's auc: 0.943982	valid_1's auc: 0.902761
[9000]	training's auc: 0.947668	valid_1's auc: 0.90263
[10000]	training's auc: 0.951231	valid_1's auc: 0.902635
Early stopping, best iteration is:
[7061]	training's auc: 0.940358	valid_1's auc: 0.903038

AUC = 0.9030379076557511

sample_num=161626
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.901218	valid_1's auc: 0.886768
[2000]	training's auc: 0.913131	valid_1's auc: 0.893927
[3000]	training's auc: 0.92083	valid_1's auc: 0.898169
[4000]	training's auc: 0.926844	valid_1's auc: 0.900356
[5000]	training's auc: 0.931901	valid_1's auc: 0.901485
[6000]	training's auc: 0.936341	valid_1's auc: 0.901986
[7000]	training's auc: 0.940516	valid_1's auc: 0.902425
[8000]	training's auc: 0.944388	valid_1's auc: 0.902703
[9000]	training's auc: 0.948107	valid_1's auc: 0.902447
[10000]	training's auc: 0.951606	valid_1's auc: 0.902597
[11000]	training's auc: 0.955017	valid_1's auc: 0.902517
Early stopping,


AUC = 0.9026087004856823

sample_num=161860
Training until validation scores don't improve for 3000 rounds.
[1000]	training's auc: 0.900799	valid_1's auc: 0.887982
[2000]	training's auc: 0.912877	valid_1's auc: 0.896398
[3000]	training's auc: 0.920928	valid_1's auc: 0.899678
[4000]	training's auc: 0.926999	valid_1's auc: 0.901534
[5000]	training's auc: 0.932262	valid_1's auc: 0.90246
[6000]	training's auc: 0.936843	valid_1's auc: 0.902734
[7000]	training's auc: 0.940976	valid_1's auc: 0.903141
[8000]	training's auc: 0.944898	valid_1's auc: 0.903214
[9000]	training's auc: 0.948632	valid_1's auc: 0.90312
[10000]	training's auc: 0.952171	valid_1's auc: 0.903191
[11000]	training's auc: 0.955465	valid_1's auc: 0.903218
[12000]	training's auc: 0.958693	valid_1's auc: 0.903065
[13000]	training's auc: 0.961777	valid_1's auc: 0.902919
Early stopping, best iteration is:
[10725]	training's auc: 0.954562	valid_1's auc: 0.903338

AUC = 0.9033382314356835

sample_num=161860
Training until validatio

In [14]:
y_test

array([[0.08171734],
       [0.21198915],
       [0.21217316],
       ...,
       [0.00286547],
       [0.06510396],
       [0.06218417]])

In [15]:
#結果保存
if (submit_flg ==1 and sampling_flg==0):
    sample = pd.read_csv(sample_submission_dir)
    sample.target = y_test
    sample.ID_code = test_df['ID_code']
    sample.to_csv(test_preds_dir, index=False)
    